In [33]:
from ray.rllib.offline.json_reader import JsonReader
import numpy as np
import tensorflow as tf
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

In [34]:
input_reader = JsonReader("logs/APPO/RE3")

2023-01-03 11:29:04,952	WARNING json_reader.py:238 -- Treating input directory as glob patterns: ['/tf/Cage/Notebooks/logs/APPO/RE3/*.json', '/tf/Cage/Notebooks/logs/APPO/RE3/*.zip']


In [35]:
sequence_length=3
num_episodes = 10000
windows_per_episode = 100-sequence_length
state_len = 52
num_action = 1
encoding_len = state_len + num_action
state_to_index = {}
index_to_state = {}
state_index = 0

data_points = num_episodes * windows_per_episode

X = np.zeros((data_points,sequence_length,encoding_len))
Y = np.zeros(data_points)

index = 0
for e in trange(num_episodes):
    data = input_reader.next()
    data['obs'].shape
    for i in range(0, windows_per_episode):
        if not data['obs'][i+1].tobytes() in state_to_index:
            state_to_index[data['obs'][i+1].tobytes()] = state_index
            index_to_state[state_index] = data['obs'][i+1].tobytes()
            state_index += 1
        r = data['rewards'][i]
        if i < sequence_length:
            s = np.zeros((sequence_length, encoding_len))
            for k in range(0,i):
                s[k,:] = np.concatenate([data['obs'][k], np.zeros(num_action)])
        else:
            #vec = np.zeros((sequence_length, num_action))
            #vec[np.arange(sequence_length),data['actions'][i-sequence_length:i]] = 1
            vec = np.array(data['actions'][i-sequence_length:i]) / 145
            vec = vec.reshape(sequence_length,1)
            s = np.concatenate([data['obs'][i-sequence_length:i], vec], axis=1)
            
        X[index,:,:] = s
        Y[index] = r
        index += 1
index

100%|██████████| 10000/10000 [03:19<00:00, 50.18it/s]


970000

In [37]:
from keras.layers import Bidirectional, Input
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import Bidirectional


model = Sequential()
model.add(Bidirectional(LSTM(256, activation='relu', return_sequences=True), input_shape=(sequence_length, encoding_len)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

print(model.summary())

model.fit(X, Y, epochs=250, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=512)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 3, 512)           634880    
 nal)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 1536)              0         
                                                                 
 dense_48 (Dense)            (None, 256)               393472    
                                                                 
 dense_49 (Dense)            (None, 128)               32896     
                                                                 
 dense_50 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,061,377
Trainable params: 1,061,377
Non-trainable params: 0
___________________________________________

In [9]:
sequence_length=1
num_episodes = 10000
windows_per_episode = 100-sequence_length
state_len = 52

data_points = num_episodes * windows_per_episode

X = np.zeros((data_points,state_len))
Y = np.zeros(data_points)

index = 0
for e in trange(num_episodes):
    data = input_reader.next()
    for i in range(0, 100):
        r = data['rewards'][i]
        s = data['obs'][i]
            
        X[index,:] = s
        Y[index] = r
        index += 1

 99%|█████████▉| 9900/10000 [02:41<00:01, 61.38it/s]


IndexError: index 990000 is out of bounds for axis 0 with size 990000

In [24]:
X.shape

(990000, 52)

In [7]:
input_reader.next()['obs'].shape

(100, 52)

In [ ]:
val loss = 0.054

In [30]:
from keras.layers import Bidirectional, Input
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import Bidirectional


model = Sequential()
#model.add(Bidirectional(LSTM(256, activation='relu', return_sequences=True), input_shape=(sequence_length, state_len)))
#model.add(Flatten())
model.add(Input(52))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

print(model.summary())

model.fit(X, Y, epochs=250, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=512)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_39 (Dense)            (None, 256)               13568     
                                                                 
 dense_40 (Dense)            (None, 256)               65792     
                                                                 
 dense_41 (Dense)            (None, 256)               65792     
                                                                 
 dense_42 (Dense)            (None, 256)               65792     
                                                                 
 dense_43 (Dense)            (None, 128)               32896     
                                                                 
 dense_44 (Dense)            (None, 1)                 129       
                                                                 
Total params: 243,969
Trainable params: 243,969
Non-t

In [31]:
predictions = model.predict(X[0:100])
count = 0
for i in range(100):
    print(str(predictions[i]) + ' : ' + str(Y[i]))

[-0.15961137] : 0.0
[-0.11099818] : 0.0
[-0.10148653] : -0.10000000149011612
[-0.1171312] : -0.10000000149011612
[-0.23810643] : -0.10000000149011612
[-0.18244702] : -0.10000000149011612
[-0.22355002] : -0.10000000149011612
[-0.12669125] : -0.10000000149011612
[-0.46990216] : -0.10000000149011612
[-0.36177063] : -0.10000000149011612
[-0.6104989] : -0.10000000149011612
[-0.12052961] : -0.10000000149011612
[-0.21812779] : -0.10000000149011612
[-0.17876002] : -0.10000000149011612
[-0.21812779] : -0.10000000149011612
[-0.12052961] : -0.10000000149011612
[-0.23810643] : -0.10000000149011612
[-0.21812779] : -0.10000000149011612
[-1.3654217] : -0.10000000149011612
[-0.1171312] : -0.10000000149011612
[-0.19984646] : -0.10000000149011612
[-0.21812779] : -0.10000000149011612
[-0.2226271] : -0.10000000149011612
[-0.12827392] : -0.10000000149011612
[-0.44063365] : -0.10000000149011612
[-0.354869] : -0.10000000149011612
[-0.23865794] : -0.10000000149011612
[-0.88061064] : -0.10000000149011612
[-0.4

In [23]:
model.save_weights('RewardModel')

In [28]:
model.predict(np.zeros((1,52)))

array([[-0.1508883]], dtype=float32)